In [ ]:
import os
import pickle

import requests
import yaml
import pandas as pd

with open("config.yml") as ymlfile:
    config = yaml.safe_load(ymlfile)
SERP_API_KEY = config.get("SERP_API_KEY")
query = "Python developer"
url = f"https://serpapi.com/search.json?engine=google_jobs&q={query}&api_key={SERP_API_KEY}&hl=en&ltype=1"
data = []
if os.path.exists("cache.pkl"):
    with open("cache.pkl", "rb") as f:
        data = pickle.load(f)
else:
    response = requests.get(url)
    data.append(response.json())
    while response.json().get("serpapi_pagination"):
        next_page_token = response.json().get("serpapi_pagination").get("next_page_token")
        url = f"https://serpapi.com/search.json?engine=google_jobs&q={query}&api_key={SERP_API_KEY}&hl=en&ltype=1&next_page_token={next_page_token}"
        response = requests.get(url)
        data.append(response.json())
    with open("cache.pkl", "wb") as f:
        pickle.dump(data, f)


In [ ]:
data_df = pd.json_normalize([item for sublist in data for item in sublist.get("jobs_results", [])])
data_df.head()

In [20]:
cloude_text = "\n".join(data_df.description.values)
with open("job_descriptions.txt", "w") as f:
    f.write(cloude_text)